In [7]:
import getpass
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
input_msg = input("請輸入問題")
result = model.invoke(
    [
        SystemMessage(content="請使用繁體中文回答。"),
        HumanMessage(content=input_msg),
    ]
)
display(Markdown(result.content))



e:\anaconda3\envs\AI\Lib\site-packages\langchain_google_genai\chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AI，全稱**人工智慧**，是一種模擬人類智慧的技術。它涵蓋了許多不同的領域，例如：

* **機器學習 (Machine Learning)**：讓電腦從數據中學習，並自動改進其表現，例如：辨識圖片、預測天氣。
* **深度學習 (Deep Learning)**：一種更複雜的機器學習，使用神經網路來處理大量數據，例如：自動翻譯、語音辨識。
* **自然語言處理 (Natural Language Processing)**：讓電腦理解和處理人類語言，例如：聊天機器人、自動摘要。
* **電腦視覺 (Computer Vision)**：讓電腦像人類一樣「看」和「理解」影像，例如：自動駕駛、影像辨識。
* **機器人學 (Robotics)**：設計和製造機器人，讓它們能夠執行各種任務，例如：自動化生產、醫療輔助。

**簡單來說，AI就是讓電腦像人類一樣思考和行動的技術。**它可以幫助我們解決各種問題，例如：提高生產效率、改善醫療保健、促進科學研究等等。

**AI的發展還在不斷進步，未來將會在更多領域發揮作用。**


In [8]:
from langchain_core.runnables import RunnableLambda
from functools import partial
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [10]:
## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
instruct_chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser() # stop the output

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print(zsc_call("Should I take the next exit, or keep going to the next one?"))

car


In [11]:
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | PPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

State:  {'topic': 'boat'}


'The old, weathered boat, its hull a mosaic of barnacles and memories, whispered tales of distant shores and forgotten dreams as it bobbed gently on the waves. \n'